# Generación de gráficas a nivel federal

En la siguiente libreta se extraerán datos de Datos Abiertos Bases Históricas de Enfermedades Transmitidas por Vector y se generaran algunas gráficas esenciales.

In [ ]:
# Importamos librerías necesarias

import pandas as pd
import sqlite3

In [121]:
# Realizamos la conexión con las base de datos

connection = sqlite3.connect("database/enfermedades_db.db")
print (connection.total_changes)
cursor = connection.cursor()


0


In [122]:
# Escribimos una query inicial, uniendo las tablas dictamen, estatus_caso y resultado_pcr
query = '''
SELECT ID_REGISTRO, SEXO, EDAD_ANOS, ENTIDAD_RES, DICTAMEN, RESULTADO_PCR, ESTATUS_CASO, dictamen.DESCRIPCIÓN, estatus_caso.DESCRIPCIÓN, resultado_pcr.DESCRIPCIÓN 
FROM registro 
INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave 
INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave 
INNER JOIN resultado_pcr ON registro.RESULTADO_PCR = resultado_pcr.clave;
'''

In [123]:
df = pd.read_sql_query(query, connection)
df.head()

,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,DICTAMEN,RESULTADO_PCR,ESTATUS_CASO,DESCRIPCIÓN,DESCRIPCIÓN,DESCRIPCIÓN
0,852727,1,50,16,5,5,3,NO APLICA,DESCARTADO,SIN SEROTIPO AISLADO
1,1001218,2,17,6,5,5,3,NO APLICA,DESCARTADO,SIN SEROTIPO AISLADO
2,1001227,2,2,31,5,5,1,NO APLICA,PROBABLE,SIN SEROTIPO AISLADO
3,1001243,1,69,20,5,5,1,NO APLICA,PROBABLE,SIN SEROTIPO AISLADO
4,1001248,1,22,27,5,3,2,NO APLICA,CONFIRMADO,DENV3


In [126]:
lista = list(df.columns)
nuevas_columnas = lista[0:-3] + ['Descripcion_dictamen', 'Descripcion_estatus', 'Descripcion_pcr']
df.columns = nuevas_columnas
df.head()

,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,DICTAMEN,RESULTADO_PCR,ESTATUS_CASO,Descripcion_dictamen,Descripcion_estatus,Descripcion_pcr
0,852727,1,50,16,5,5,3,NO APLICA,DESCARTADO,SIN SEROTIPO AISLADO
1,1001218,2,17,6,5,5,3,NO APLICA,DESCARTADO,SIN SEROTIPO AISLADO
2,1001227,2,2,31,5,5,1,NO APLICA,PROBABLE,SIN SEROTIPO AISLADO
3,1001243,1,69,20,5,5,1,NO APLICA,PROBABLE,SIN SEROTIPO AISLADO
4,1001248,1,22,27,5,3,2,NO APLICA,CONFIRMADO,DENV3


La base de datos no solo tiene entradas o renglones para pacientes con dengue confirmado. Por ejemplo, en dictamen tenemos valores como 'NEGATIVO' y en estatus_caso se tiene 'DESCARTADO' Y 'PROBABLE'. Ante esto, nos enfocaremos en los dictamenes con 'CONFIRMADO A DENGUE', 'NO APLICA' y 'EN ESTUDIO' y con estatus 'CONFIRMADO'.

In [128]:
dictamenes_aceptables = ['CONFIRMADO A DENGUE', 'NO APLICA', 'EN ESTUDIO']
estatus_aceptable = 'CONFIRMADO'

df_aceptable = df[(df['Descripcion_dictamen'].isin(dictamenes_aceptables)) &  (df['Descripcion_estatus'] == 'CONFIRMADO')]
df_aceptable.head()

,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,DICTAMEN,RESULTADO_PCR,ESTATUS_CASO,Descripcion_dictamen,Descripcion_estatus,Descripcion_pcr
4,1001248,1,22,27,5,3,2,NO APLICA,CONFIRMADO,DENV3
27,1001344,2,10,27,5,3,2,NO APLICA,CONFIRMADO,DENV3
29,1001315,1,40,30,5,5,2,NO APLICA,CONFIRMADO,SIN SEROTIPO AISLADO
32,1001379,1,3,6,5,2,2,NO APLICA,CONFIRMADO,DENV2
33,1001373,1,9,6,5,2,2,NO APLICA,CONFIRMADO,DENV2


Obtendremos primero la porporción de los serotipos de Dengue registrados en esta subcolección de datos.

In [140]:
df_pcr = pd.DataFrame(df_aceptable.groupby('Descripcion_pcr').size())
df_pcr.reset_index(inplace = True)
df_pcr.columns = ['Descripcion_pcr', 'Frecuencia']
df_pcr.head()

,Descripcion_pcr,Frecuencia
0,DENV1,1054
1,DENV2,1409
2,DENV3,2477
3,DENV4,130
4,SIN SEROTIPO AISLADO,3425


In [141]:
import plotly.express as px
fig = px.pie(df_pcr, values='Frecuencia', names = 'Descripcion_pcr', title = 'Serotipos de Dengue')
fig.update_layout(
    width=700,
    height=500,
)
fig.show()

Intentaremos ahora ver la proporción de personas afectadas tomando como referencia el sexo.

In [143]:
df_sexo = pd.DataFrame(df_aceptable.groupby('SEXO').size()).reset_index()
df_sexo

,SEXO,0
0,1,4517
1,2,3978


In [144]:
df_sexo['Sexo'] = ['Mujer', 'Hombre']
df_sexo.columns = ['Clave', 'Frecuencia', 'Sexo']

In [146]:
fig = px.bar(df_sexo, x='Sexo', y='Frecuencia', title = 'Proporción de afectados por sexo')
fig.update_layout(
    width=700,
    height=500,
)
fig.show()

Por otro lado, debido la gran variabilidad climática dentro del país, resulta interesante preguntarse que entidades son las que más son afectadas por el dengue.

In [153]:
df_entidad =  pd.DataFrame(df_aceptable.groupby('ENTIDAD_RES').size()).reset_index()
df_entidad.head()

,ENTIDAD_RES,0
0,1,1
1,2,1
2,3,23
3,4,91
4,5,5


Para obtener la descripción de la clave de la entidad, realizaremos otra solicitud básica con sql.

In [148]:
df_entidad_clave = pd.read_sql_query("SELECT * FROM entidad", connection)
df_entidad_clave.head()

,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,AGUASCALIENTES,AS
1,2,BAJA CALIFORNIA,BC
2,3,BAJA CALIFORNIA SUR,BS
3,4,CAMPECHE,CC
4,5,COAHUILA DE ZARAGOZA,CL


Cambiamos el nombre de las columnas de df_entidad para poder realizar un join.

In [149]:
df_entidad.columns= ['CLAVE_ENTIDAD','Frecuencia']
df_entidad_descripcion = pd.merge(df_entidad, df_entidad_clave, 'inner', on = 'CLAVE_ENTIDAD')
# Ordenamos los datos por frecuencia de mayor a menor
df_entidad_descripcion.sort_values(by = 'Frecuencia', ascending = True, inplace = True)

In [151]:
fig = px.bar(df_entidad_descripcion, x="Frecuencia", y="ENTIDAD_FEDERATIVA", orientation='h', title = 'Dengue por estado')
fig.update_layout(
    width=700,
    height=500,
)
fig.show()

También resulta interesante preguntarse que grupo de edad es el más afectado por dengue.

In [154]:
fig = px.histogram(df_aceptable, x="EDAD_ANOS", nbins=10, title = 'Histograma de frecuencia de dengue por edad')
fig.update_layout(
    width=700,
    height=500,
)
fig.show()

Con estas gráficas hechas, cerramos la conexión con la base de datos.

In [155]:
connection.close()